<a href="https://colab.research.google.com/github/dotsnangles/BART-Text-Summarisation/blob/main/chatbot_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !rm -rf /content/Poly-Encoder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!gdown -q --folder 1Ipr-aNF5ELMY0HTXAmeV26LlgktKUfmG

In [ ]:
!nvidia-smi

Wed Jul  6 00:10:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -q transformers datasets folium==0.2.1

     |████████████████████████████████| 4.4 MB 4.3 MB/s 
     |████████████████████████████████| 362 kB 47.5 MB/s 
     |████████████████████████████████| 69 kB 9.0 MB/s 
     |████████████████████████████████| 596 kB 59.2 MB/s 
     |████████████████████████████████| 101 kB 12.9 MB/s 
     |████████████████████████████████| 6.6 MB 51.4 MB/s 
     |████████████████████████████████| 212 kB 56.8 MB/s 
     |████████████████████████████████| 1.1 MB 56.2 MB/s 
     |████████████████████████████████| 140 kB 57.9 MB/s 
     |████████████████████████████████| 127 kB 65.5 MB/s 
     |████████████████████████████████| 271 kB 58.7 MB/s 
     |████████████████████████████████| 144 kB 67.3 MB/s 
     |████████████████████████████████| 94 kB 3.4 MB/s 


In [ ]:
!git clone https://github.com/dotsnangles/Poly-Encoder.git

Cloning into 'Poly-Encoder'...
remote: Enumerating objects: 129, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 129 (delta 41), reused 20 (delta 8), pack-reused 57
Receiving objects: 100% (129/129), 40.70 KiB | 1.07 MiB/s, done.
Resolving deltas: 100% (66/66), done.


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertPreTrainedModel, BertConfig, BertModel, BertTokenizer, AutoModel
from encoder import PolyEncoder
from transform import SelectionJoinTransform, SelectionSequentialTransform

PATH = '/content/Poly-Encoder/chatbot_output/poly_16_pytorch_model.bin'

bert_name = 'klue/bert-base'
bert_config = BertConfig.from_pretrained(bert_name)

tokenizer = BertTokenizer.from_pretrained(bert_name)
tokenizer.add_tokens(['\n'], special_tokens=True)

context_transform = SelectionJoinTransform(tokenizer=tokenizer, max_len=256)
response_transform = SelectionSequentialTransform(tokenizer=tokenizer, max_len=128)

bert = BertModel.from_pretrained(bert_name, config=bert_config)

model = PolyEncoder(bert_config, bert=bert, poly_m=16)
model.resize_token_embeddings(len(tokenizer))
# model.load_state_dict(torch.load(PATH))

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Embedding(32001, 768)

In [ ]:
context = ['This framework generates embeddings for each input sentence', 
           'Sentences are passed as a list of string.', 
           'The quick brown fox jumps over the lazy dog.']

candidates = ['This framework generates embeddings for each input sentence']

In [ ]:
context_input_ids, context_input_masks = context_transform(context)
responses_token_ids_list, responses_input_masks_list = response_transform(candidates)

In [ ]:
contexts_token_ids_list_batch, contexts_input_masks_list_batch, \
responses_token_ids_list_batch, responses_input_masks_list_batch = [context_input_ids], [context_input_masks], [responses_token_ids_list], [responses_input_masks_list]

long_tensors = [contexts_token_ids_list_batch, contexts_input_masks_list_batch,
                                            responses_token_ids_list_batch, responses_input_masks_list_batch]

contexts_token_ids_list_batch, contexts_input_masks_list_batch, \
responses_token_ids_list_batch, responses_input_masks_list_batch = (torch.tensor(t, dtype=torch.long) for t in long_tensors)

In [ ]:
# forward
model(contexts_token_ids_list_batch, contexts_input_masks_list_batch, responses_token_ids_list_batch, responses_input_masks_list_batch)

In [ ]:
def embs_gen(contexts_token_ids_list_batch, contexts_input_masks_list_batch):

    with torch.no_grad():
        model.eval()
        
        ctx_out = model.bert(contexts_token_ids_list_batch, contexts_input_masks_list_batch)[0]  # [bs, length, dim]
        poly_code_ids = torch.arange(model.poly_m, dtype=torch.long).to(contexts_token_ids_list_batch.device)
        poly_code_ids = poly_code_ids.unsqueeze(0).expand(1, model.poly_m)
        poly_codes = model.poly_code_embeddings(poly_code_ids) # [bs, poly_m, dim]
        embs = model.dot_attention(poly_codes, ctx_out, ctx_out) # [bs, poly_m, dim]

        return embs

NameError: ignored

In [ ]:
embs = embs_gen(contexts_token_ids_list_batch, contexts_input_masks_list_batch)

In [ ]:
def cand_emb_gen(responses_token_ids_list_batch, responses_input_masks_list_batch):

    with torch.no_grad():
        model.eval()
                
        batch_size, res_cnt, seq_length = responses_token_ids_list_batch.shape # res_cnt is 1 during training
        responses_token_ids_list_batch = responses_token_ids_list_batch.view(-1, seq_length)
        responses_input_masks_list_batch = responses_input_masks_list_batch.view(-1, seq_length)
        cand_emb = model.bert(responses_token_ids_list_batch, responses_input_masks_list_batch)[0][:,0,:] # [bs, dim]
        cand_emb = cand_emb.view(batch_size, res_cnt, -1) # [bs, res_cnt, dim]

        return cand_emb

In [ ]:
cand_emb = cand_emb_gen(responses_token_ids_list_batch, responses_input_masks_list_batch)

In [ ]:
def score(embs, cand_emb):
    with torch.no_grad():
        model.eval()

        ctx_emb = model.dot_attention(cand_emb, embs, embs) # [bs, res_cnt, dim]
        dot_product = (ctx_emb*cand_emb).sum(-1)
        
        return dot_product

In [ ]:
score(embs, cand_emb)

In [ ]:
embs.shape

torch.Size([1, 16, 768])

In [ ]:
model.dot_attention()